## Define the item response model

The probability for a student with skill $s$ to answer correctly a question of difficulty $d$ is:

$$P(X=1 | s, d) = \sigma(s-d)$$


In [ ]:
import torch
import torch.nn.functional as F
from torch.distributions import Bernoulli
from tqdm import trange
import matplotlib.pyplot as plt

class ResponseModel(torch.nn.Module):
    def __init__(self, n_students, n_questions):
        super().__init__()
        self.skill = torch.nn.Parameter(torch.zeros(n_students))
        self.difficulty = torch.nn.Parameter(torch.zeros(n_questions))

    def forward(self):
        return Bernoulli(probs=torch.sigmoid(self.skill[:, None] - self.difficulty[None, :]))

## Generate some data


In [ ]:
def generate_data(n_students, n_questions):
    # Sample students, questions, and answers
    model = ResponseModel(n_students, n_students)
    model.skill.data = -3 + 6*torch.rand(n_students)
    model.difficulty.data = -3 + 6*torch.rand(n_questions)
    answers = model().sample()
    
    # Pick a cheater, and make their answer correct with probability 1/2
    cheater = torch.randint(high=n_students, size=(1,)).item()
    is_cheating = Bernoulli(probs=0.5*torch.ones(n_questions)).sample().bool()
    answers[cheater] = torch.where(is_cheating, torch.ones(n_questions), answers[cheater])
    return model, answers, cheater

true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
print("Skill", true_model.skill)
print("Difficulty", true_model.difficulty)
print("Answers", answers)

## Fit the model by Maximum Likelihood Estimation
Optimized by Stochastic Gradient Descent

In [ ]:
def fit(model, answers, epochs, alpha=1.0, see_progress=True):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    if torch.cuda.is_available():
        model, answers = model.cuda(), answers.cuda()
    losses = []
    for _ in trange(epochs) if see_progress else range(epochs):
        optimizer.zero_grad()
        distribution = model()
        loss = -distribution.log_prob(answers).mean() + alpha*torch.pow(model.difficulty.mean(), 2)
        losses.append(loss.detach())
        loss.backward()
        optimizer.step()
    model, answers = model.cpu(), answers.cpu()
    return model, losses

model = ResponseModel(n_students=answers.shape[0], n_questions=answers.shape[1])
model, losses = fit(model, answers, epochs=1000, alpha=1e-2)
plt.plot(losses, label="loss")
plt.yscale("log")
plt.xlabel("epochs")
plt.legend()

## Visualize estimation errors

In [ ]:
plt.hist((true_model.difficulty - model.difficulty).detach().numpy(), density=True, label="difficulty error")
plt.hist((true_model.skill - model.skill).detach().numpy(), density=True, label="skill error")
plt.legend()

You may notice that one single student has a high skill error... Who could that be?

## Predict the cheater, with lowest answers likelihood

In [ ]:
log_probs = model().log_prob(answers).sum(axis=1).detach().numpy()
predicted_cheater = log_probs.argmin()
print(f"Predicted cheater: {predicted_cheater}. True cheater: {cheater}")
print(f"Predicted skill: {model.skill[predicted_cheater]}, true skill: {true_model.skill[cheater]}")

plt.scatter(range(log_probs.size), log_probs, marker='.')
plt.scatter(x=cheater, y=log_probs[cheater], c='r', marker='o', label="true cheater")
plt.scatter(x=predicted_cheater, y=log_probs[predicted_cheater], marker='o', s=80,
            facecolor='none', edgecolor='b', label="predicted cheater")
plt.ylabel("log prob of answers")
plt.xlabel("student")
plt.legend()

## Repeat and measure success rate
*It should be enabled by default, but be sure to select GPU in* `Execution / Execution Type` *to get a faster training time (~4mn).* 

In [ ]:
samples = 100
successes = 0
for _ in trange(samples):
    true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
    model = ResponseModel(n_students=answers.shape[0], n_questions=answers.shape[1])
    model, _ = fit(model, answers, epochs=1000, alpha=1e-2, see_progress=False)
    predicted_cheater = model().log_prob(answers).sum(axis=1).detach().numpy().argmin()
    successes += predicted_cheater == cheater
print()
print(f"Success rate: {successes/samples*100:.1f}%")


Unfortunately, the success rate is too low, and does not reach the desired target of 84%.
This is because our response model does not account for the presence of the cheater, and its effects on the distribution of answers.

Let us try to include this information in our model.

## Include the cheater in the item response model

We add an additional parameter to the model: the (log-)probability that any student is the cheater.  

In [ ]:
class ResponseModelWithCheater(ResponseModel):
    def __init__(self, n_students, n_questions):
        super().__init__(n_students, n_questions)
        self.cheater_logits = torch.nn.Parameter(torch.zeros(n_students))

    def forward(self):
        cheater_prob = F.softmax(self.cheater_logits)[:, None]
        answer_no_cheat = torch.sigmoid(self.skill[:, None] - self.difficulty[None, :])
        answer_cheat = 0.5 * torch.sigmoid(self.skill[:, None] - self.difficulty[None, :]) + 0.5 * 1
        return Bernoulli(probs=cheater_prob*answer_cheat + (1-cheater_prob)*answer_no_cheat)


## Fit the updated model with cheater

In [ ]:
model = ResponseModelWithCheater(n_students=answers.shape[0], n_questions=answers.shape[1])
model, losses = fit(model, answers, epochs=1000, alpha=1e-2)
plt.plot(losses, label="loss")
plt.yscale("log")
plt.xlabel("epochs")
plt.legend()

## Plot the cheater probability

In [ ]:
cheater_probs = F.softmax(model.cheater_logits).detach().numpy()
plt.axvline(x=cheater, c='r')
plt.plot(cheater_probs)
plt.xlabel("student")
plt.ylabel("cheater probability")

## Repeat and measure success rate

In [ ]:
samples = 100
successes = 0
for _ in trange(samples):
    true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
    model = ResponseModelWithCheater(n_students=answers.shape[0], n_questions=answers.shape[1])
    model, _ = fit(model, answers, epochs=1000, alpha=1e-2, see_progress=False)
    predicted_cheater = F.softmax(model.cheater_logits).detach().numpy().argmax()
    successes += predicted_cheater == cheater
print()
print(f"Success rate: {successes/samples*100:.1f}%")
